In [1]:
2+3


5

In [1]:
import Flux

┌ Warning: The NVIDIA driver on this system only supports up to CUDA 10.2.0.
│ For performance reasons, it is recommended to upgrade to a driver that supports CUDA 11.2 or higher.
└ @ CUDA C:\Users\anhin\.julia\packages\CUDA\mVgLI\src\initialization.jl:42


In [2]:
f(x)=max(0,x)

f (generic function with 1 method)

In [16]:
x = [3f0, -4f0, 8f0]

3-element Vector{Float32}:
  3.0
 -4.0
  8.0

In [5]:
using Flux

In [17]:
p = params(x)

Params([Float32[3.0, -4.0, 8.0]])

In [18]:
map(f, x)

3-element Vector{Float32}:
 3.0
 0.0
 8.0

In [19]:
sum(map(f, x))

11.0f0

In [20]:
grads = gradient(()->sum(map(f, x)), p)

Grads(...)

In [21]:
grads[x]

3-element Vector{Float32}:
 1.0
 0.0
 1.0

In [22]:
pars = Dict(:x=>0f0, "y"=>4f0, 8=>-3f0)

Dict{Any, Float32} with 3 entries:
  "y" => 4.0
  8   => -3.0
  :x  => 0.0

In [24]:
keys(pars)

KeySet for a Dict{Any, Float32} with 3 entries. Keys:
  "y"
  8
  :x

In [25]:
Dict(k=>f(pars[k]) for k in keys(pars))

Dict{Any, Float32} with 3 entries:
  :x  => 0.0
  8   => 0.0
  "y" => 4.0

In [26]:
pd = params(pars)

Params([])

In [28]:
values(pars)

ValueIterator for a Dict{Any, Float32} with 3 entries. Values:
  4.0f0
  -3.0f0
  0.0f0

In [30]:
sum(values(Dict(k=>f(pars[k]) for k in keys(pars))))

4.0f0

In [31]:
grads = gradient(()->sum(values(Dict(k=>f(pars[k]) for k in keys(pars)))), pd)

LoadError: MethodError: no method matching iterate(::ErrorException)
[0mClosest candidates are:
[0m  iterate([91m::Union{LinRange, StepRangeLen}[39m) at range.jl:664
[0m  iterate([91m::Union{LinRange, StepRangeLen}[39m, [91m::Int64[39m) at range.jl:664
[0m  iterate([91m::T[39m) where T<:Union{Base.KeySet{var"#s79", var"#s78"} where {var"#s79", var"#s78"<:Dict}, Base.ValueIterator{var"#s77"} where var"#s77"<:Dict} at dict.jl:693
[0m  ...

In [33]:
# test an example from https://discourse.julialang.org/t/list-comprehension-in-zygote/41881/7

w = randn(5)
grads = gradient(params(w)) do
    dict = Dict{Int, Float64}(i => v for (i,v) in enumerate(w))
    # m = maximum(i*v for (i,v) in dict) # doesn't work
    m = maximum(i -> i * dict[i], eachindex(w)) # works
    k = randn(5)
    sum(k .- m)
end

Grads(...)

In [34]:
grads[w]

5-element Vector{Float64}:
   0.0
   0.0
   0.0
 -20.0
   0.0

In [35]:
w

5-element Vector{Float64}:
 -1.1023486804734226
 -0.3668550123369823
  0.9043130209527922
  1.8999952767174157
 -0.28891773863642767

In [36]:
enumerate(w)

enumerate([-1.1023486804734226, -0.3668550123369823, 0.9043130209527922, 1.8999952767174157, -0.28891773863642767])

In [40]:
dict = Dict{Int, Float64}(i => v for (i,v) in enumerate(w))

Dict{Int64, Float64} with 5 entries:
  5 => -0.288918
  4 => 1.9
  2 => -0.366855
  3 => 0.904313
  1 => -1.10235

In [38]:
eachindex(w)

Base.OneTo(5)

In [41]:
m = maximum(i*v for (i,v) in dict) # gradient doesn't work

7.599981106869663

In [42]:
m = maximum(i -> i * dict[i], eachindex(w)) # gradient works

7.599981106869663

In [43]:
k = randn(5)

5-element Vector{Float64}:
 -0.7872126582020299
 -0.14606729956981565
 -2.308545916020537
 -1.2319227561813213
  2.4929111672818425

In [44]:
sum(k .- m)

-39.98074299704017

In [48]:
# rewriting sum(values(Dict(k=>f(pars[k]) for k in keys(pars))))

function sum_relu_dict(d)
    s = 0f0
    for k in keys(d)
        s += f(d[k])
    end
    s
end

sum_relu_dict (generic function with 1 method)

In [49]:
sum_relu_dict(pars)

4.0f0

In [50]:
grads = gradient(()->sum_relu_dict(pars), pd)

Grads(...)

In [51]:
grads[pars]

Dict{Any, Any} with 3 entries:
  "y" => 1.0
  8   => 0.0
  :x  => 1.0